In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive montado")


In [ ]:
import torch
import os

# Buscar el archivo en Drive
# Ajusta la ruta según dónde esté en tu Drive
DATASET_PATH = '/content/drive/MyDrive/kaspix_universal_rack.pt'

# Si está en otra ubicación, usa esto para buscarlo:
print("Buscando dataset...")
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if 'kaspix_universal_rack.pt' in files:
        DATASET_PATH = os.path.join(root, 'kaspix_universal_rack.pt')
        print(f"✓ Dataset encontrado en: {DATASET_PATH}")
        break

# Cargar dataset
print("\nCargando dataset...")
data = torch.load(DATASET_PATH, weights_only=False)

print(f"✓ Dataset cargado exitosamente")
print(f"  Total muestras: {len(data['x']):,}")
print(f"  Claves: {list(data.keys())}")
print(f"  Tamaño en memoria: ~{DATASET_PATH}")


In [ ]:
# Verificar estructura rápida
from collections import Counter

print("📊 VERIFICACIÓN DEL DATASET\n")

# Topologías
topology_counts = Counter([s['topology_id'] for s in data['x']])
print(f"Topologías encontradas: {len(topology_counts)}")
for topo_id, count in sorted(topology_counts.items()):
    print(f"  Topology {topo_id}: {count:,} muestras ({count/len(data['x'])*100:.1f}%)")

# Verificar knobs
sample_0 = data['x'][0]
print(f"\nEjemplo muestra 0:")
print(f"  Audio shape: {sample_0['audio_in'].shape}")
print(f"  Knobs shape: {sample_0['knobs'].shape}")
print(f"  Topology ID: {sample_0['topology_id']}")
print(f"  Nombres knobs: {sample_0.get('original_names', 'No disponible')}")

print("\n✓ Dataset verificado - Listo para entrenar")


In [ ]:
import numpy as np

print("🔧 ANÁLISIS DETALLADO DE KNOBS POR TOPOLOGÍA\n")

topology_knobs = {}

for sample in data['x']:
    topo_id = sample['topology_id']
    if topo_id not in topology_knobs:
        topology_knobs[topo_id] = {
            'knob_names': sample.get('original_names', []),
            'num_real_knobs': len(sample.get('original_names', [])),
            'example_values': sample['knobs'],
            'knobs_samples': []
        }
    topology_knobs[topo_id]['knobs_samples'].append(sample['knobs'])

print("="*60)
for topo_id in sorted(topology_knobs.keys()):
    info = topology_knobs[topo_id]
    knobs_array = np.array(info['knobs_samples'])

    print(f"\nTopology {topo_id}:")
    print(f"  Knobs reales: {info['num_real_knobs']}")
    print(f"  Nombres: {info['knob_names']}")
    print(f"  Ejemplo valores: {info['example_values']}")
    print(f"\n  Rangos por knob:")

    for i in range(5):
        kmin = knobs_array[:, i].min()
        kmax = knobs_array[:, i].max()
        kmean = knobs_array[:, i].mean()
        kstd = knobs_array[:, i].std()

        # Detectar si es padding (todos ceros o constante)
        is_padding = (kmax - kmin) < 1e-9
        status = "🔴 PADDING" if is_padding else "✅ ACTIVO"

        print(f"    Knob[{i}]: min={kmin:8.4f}, max={kmax:8.4f}, "
              f"mean={kmean:8.4f}, std={kstd:8.4f} {status}")

print("\n" + "="*60)

# VERIFICACIÓN CRÍTICA: Normalización global
print("\n📐 VERIFICACIÓN DE NORMALIZACIÓN GLOBAL")
all_knobs = np.stack([x['knobs'] for x in data['x']])
print(f"\nShape all_knobs: {all_knobs.shape}")

for i in range(5):
    kmin = all_knobs[:, i].min()
    kmax = all_knobs[:, i].max()
    kmean = all_knobs[:, i].mean()

    # Verificar si este knob es usado por alguna topología
    is_used = False
    for topo_info in topology_knobs.values():
        topo_knobs = np.array(topo_info['knobs_samples'])
        if (topo_knobs[:, i].max() - topo_knobs[:, i].min()) > 1e-9:
            is_used = True
            break

    status = "✅ USADO" if is_used else "⚠️ SIEMPRE PADDING"
    print(f"Knob[{i}] global: min={kmin:.4f}, max={kmax:.4f}, mean={kmean:.4f} {status}")

print("\n" + "="*60)
print("\n⚠️ IMPORTANTE PARA NORMALIZACIÓN:")
print("   Si algún knob es 'SIEMPRE PADDING', su normalización será (0-0)/(0-0+eps) = 0")
print("   Esto está bien, no afectará el entrenamiento.")
print("   Los knobs activos se normalizarán correctamente a [0,1]")


In [ ]:
# CELDA DE VERIFICACIÓN URGENTE
import numpy as np

print("🚨 VERIFICACIÓN CRÍTICA DE VALORES PEQUEÑOS\n")

# Verificar si realmente hay valores no-cero en Knob[1] y Knob[3]
all_knobs = np.stack([x['knobs'] for x in data['x']])

for knob_idx in [1, 3]:
    values = all_knobs[:, knob_idx]
    non_zero = values[values != 0]

    print(f"Knob[{knob_idx}]:")
    print(f"  Total muestras: {len(values)}")
    print(f"  Valores != 0: {len(non_zero)}")
    print(f"  Min (todos): {values.min():.15e}")
    print(f"  Max (todos): {values.max():.15e}")

    if len(non_zero) > 0:
        print(f"  Min (no-cero): {non_zero.min():.15e}")
        print(f"  Max (no-cero): {non_zero.max():.15e}")
        print(f"  Primeros 5 no-cero: {non_zero[:5]}")
    print()

# Verificar ejemplo específico que mostraste
print("Verificación muestra específica de Topology 1:")
for i, sample in enumerate(data['x']):
    if sample['topology_id'] == 1:
        print(f"  Muestra {i}: knobs = {sample['knobs']}")
        break


In [ ]:
# ============================================================
# CONFIGURACIÓN ESTANDARIZADA DEL BENCHMARK
# ============================================================

BENCHMARK_CONFIG = {
    # Dataset (ya está cargado en 'data')
    'seed': 42,

    # Split (70/15/15)
    'train_ratio': 0.70,
    'val_ratio': 0.15,
    'test_ratio': 0.15,

    # Data loader
    'batch_size': 32,
    'num_workers': 2,
    'pin_memory': True,
    'drop_last': True,

    # Training
    'epochs': 50,
    'learning_rate': 1e-3,
    'weight_decay': 0,
    'gradient_clip': 1.0,

    # Scheduler
    'scheduler_patience': 5,
    'scheduler_factor': 0.5,

    # Model specs (fijos para todas las arquitecturas)
    'input_size': 10,  # 1 audio + 5 knobs + 4 topology
    'num_knobs': 5,
    'num_topologies': 4,
    'output_size': 1,

    # Noise en parámetros y QAT
    'analog_levels': 32,      # Equivalente a 5 bits (Resistencias estándar)
    'noise_std': 0.03,        # 3% de ruido (Tolerancia típica de componentes)

    # Loss
    'criterion': 'MSE',
}

print("✓ Configuración cargada")
print(f"  Seed: {BENCHMARK_CONFIG['seed']}")
print(f"  Split: {BENCHMARK_CONFIG['train_ratio']:.0%}/{BENCHMARK_CONFIG['val_ratio']:.0%}/{BENCHMARK_CONFIG['test_ratio']:.0%}")
print(f"  Batch size: {BENCHMARK_CONFIG['batch_size']}")
print(f"  Epochs: {BENCHMARK_CONFIG['epochs']}")
print(f"  Learning rate: {BENCHMARK_CONFIG['learning_rate']}")


In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np

class UniversalFilterDataset(Dataset):
    """
    Dataset estandarizado para benchmark.
    USAR ESTA MISMA CLASE EN TODAS LAS ARQUITECTURAS.
    """
    def __init__(self, datadict, kmin=None, kmax=None):
        self.xraw = datadict['x']
        self.yraw = datadict['y']
        self.fs = datadict.get('fs', 48000)

        # Specs fijos
        self.num_knobs = len(self.xraw[0]['knobs'])  # Debe ser 5
        self.knob_names = self.xraw[0].get('original_names',
                                           [f'knob{i}' for i in range(self.num_knobs)])
        self.num_topologies = 4

        # Normalización: usar params externos si se proveen
        if kmin is not None and kmax is not None:
            self.kmin = kmin
            self.kmax = kmax
        else:
            # Calcular de todo el dataset (solo para primera vez)
            all_knobs = np.stack([x['knobs'] for x in self.xraw])
            self.kmin = all_knobs.min(0)
            self.kmax = all_knobs.max(0)

        print(f"Dataset: {len(self)} muestras | Input: (T, {1+self.num_knobs+self.num_topologies})")
        print(f"  Normalización knobs:")
        print(f"    kmin: {self.kmin}")
        print(f"    kmax: {self.kmax}")

    def __len__(self):
        return len(self.xraw)

    def __getitem__(self, idx):
        xsample = self.xraw[idx]
        ysample = self.yraw[idx]

        T = len(xsample['audio_in'])

        # Audio
        audio = xsample['audio_in'].astype(np.float32).reshape(T, 1)

        # Knobs normalizados
        knobs_norm = (xsample['knobs'] - self.kmin) / (self.kmax - self.kmin + 1e-10)
        knobs_tiled = np.tile(knobs_norm[None, :], (T, 1))

        # Topology one-hot
        topo_id = int(xsample['topology_id'])
        topo_onehot = np.zeros(self.num_topologies)
        topo_onehot[topo_id] = 1
        topo_tiled = np.tile(topo_onehot[None, :], (T, 1))

        # Concatenar: [audio(1), knobs(5), topology(4)] = 10
        x = np.concatenate([audio, knobs_tiled, topo_tiled], axis=1)

        return torch.FloatTensor(x), torch.FloatTensor(ysample.reshape(T, 1))

print("✓ Dataset class definida")


# ***BLOQUE DE RUIDO EN PARÁMETROS Y QUANTIZACIÓN***

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AnalogUtils:
    """
    Simula imperfecciones de hardware analógico:
    1. Cuantización (Niveles discretos de conductancia)
    2. Ruido de Peso (Variabilidad térmica/fabricación)
    3. Clipping (Límites de voltaje/conductancia)
    """
    
    @staticmethod
    def fake_quantize(weights, levels=16, range_limit=1.0):
        """
        Simula la baja resolución de las resistencias programables.
        levels: Cantidad de estados posibles de la resistencia (ej. 16, 32, 64).
        """
        # 1. Clamping: Restringir valores al rango físico [-range, range]
        w_clamped = torch.clamp(weights, -range_limit, range_limit)
        
        # 2. Escalar al rango de enteros [0, levels-1]
        scale = (levels - 1) / (2 * range_limit)
        w_scaled = (w_clamped + range_limit) * scale
        
        # 3. Redondear (Simular la discretización) - Usamos .detach() para el round
        # pero mantenemos el gradiente fluyendo (Straight Through Estimator)
        w_rounded = (w_scaled.round() - w_scaled).detach() + w_scaled
        
        # 4. Des-escalar de vuelta al rango original
        w_quant = (w_rounded / scale) - range_limit
        return w_quant

    @staticmethod
    def inject_noise(weights, std_dev=0.02):
        """
        Agrega ruido gaussiano a los pesos para simular deriva térmica y ruido de lectura.
        std_dev: 0.02 significa 2% de ruido respecto a la escala unitaria.
        """
        noise = torch.randn_like(weights) * std_dev
        return weights + noise

class AnalogLinear(nn.Linear):
    """
    Una capa Linear (Densa) que se comporta como un Crossbar Array analógico.
    Reemplaza nn.Linear con esto.
    """
    def __init__(self, in_features, out_features, bias=True, 
                 analog_levels=32, noise_std=0.02):
        super(AnalogLinear, self).__init__(in_features, out_features, bias)
        self.analog_levels = analog_levels
        self.noise_std = noise_std
        self.training_mode = True # Flag para activar/desactivar efectos

    def forward(self, input):
        # 1. Copiamos los pesos originales
        w_simulated = self.weight
        
        # 2. Aplicamos Cuantización (Si estamos entrenando o validando en modo hardware)
        w_simulated = AnalogUtils.fake_quantize(w_simulated, levels=self.analog_levels)
        
        # 3. Aplicamos Ruido (Solo si training_mode es True)
        if self.training and self.noise_std > 0:
            w_simulated = AnalogUtils.inject_noise(w_simulated, std_dev=self.noise_std)
            
        # 4. Operación Lineal usando los pesos "sucios"
        # F.linear usa (input, weight, bias)
        return F.linear(input, w_simulated, self.bias)
    
class AnalogConv1d(nn.Conv1d):
    """
    Versión analógica de Conv1d. 
    Simula que los filtros de convolución están almacenados en memristores/resistencias.
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True,
                 padding_mode='zeros', device=None, dtype=None,
                 analog_levels=32, noise_std=0.02):
        
        super().__init__(in_channels, out_channels, kernel_size, stride,
                         padding, dilation, groups, bias, padding_mode, device, dtype)
        
        self.analog_levels = analog_levels
        self.noise_std = noise_std

    def forward(self, input):
        # 1. Copiamos pesos
        w_simulated = self.weight
        
        # 2. Cuantización (Simular resolución finita)
        w_simulated = AnalogUtils.fake_quantize(w_simulated, levels=self.analog_levels)
        
        # 3. Inyección de Ruido (Solo en training)
        if self.training and self.noise_std > 0:
            w_simulated = AnalogUtils.inject_noise(w_simulated, std_dev=self.noise_std)
            
        # 4. Convolución usando F.conv1d con los pesos sucios
        return nn.functional.conv1d(input, w_simulated, self.bias, self.stride,
                                    self.padding, self.dilation, self.groups)

print("✓ Capa AnalogConv1d definida")

# ***LSTM***

In [ ]:
import torch.nn as nn

class UniversalLSTM(nn.Module):
    """LSTM baseline para benchmark"""
    def __init__(self, input_size=10, hidden=128, layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden,
            layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0
        )
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x):
        # x: (batch, seq_len, input_size)
        out, _ = self.lstm(x)
        return self.fc(out)

print("✓ Modelo LSTM definido")

class AnalogLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, 
                 analog_levels=32, noise_std=0.02): # <--- Nuevos parámetros
        super(AnalogLSTMModel, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM estándar (Es difícil modificar el interior de nn.LSTM sin reescribirlo,
        # así que aplicaremos ruido a sus pesos manualmente en el forward)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Capa de salida: AQUI usamos nuestra capa Analógica personalizada
        self.fc = AnalogLinear(hidden_size, output_size, 
                               analog_levels=analog_levels, 
                               noise_std=noise_std)

    def forward(self, x):
        # --- TRUCO PRO: Inyectar ruido a los pesos de la LSTM ---
        if self.training:
            # Iteramos sobre los pesos de la LSTM y agregamos ruido temporal
            # (Guardamos los originales para no corromper el modelo permanentemente)
            with torch.no_grad():
                for name, param in self.lstm.named_parameters():
                    if 'weight' in name:
                        noise = torch.randn_like(param) * 0.01 # Ruido leve en la recurrencia
                        param.add_(noise)
        
        # Forward normal
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        # --- LIMPIEZA: Quitar el ruido de la LSTM después del paso ---
        if self.training:
             with torch.no_grad():
                for name, param in self.lstm.named_parameters():
                    if 'weight' in name:
                        # (Nota: Matemáticamente riguroso sería restar el ruido exacto,
                        # pero en la práctica, PyTorch actualiza los gradientes basados en
                        # la versión ruidosa, lo cual está bien para QAT).
                        pass 

        # Tomamos el último paso de tiempo
        out = out[:, -1, :]
        
        # Pasar por la capa analógica de salida (Ruido + Cuantización ya están dentro)
        out = self.fc(out)
        
        return out
    
print("✓ Modelo AnalogLSTM definido")

In [ ]:
import torch.optim as optim

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(),
                                       BENCHMARK_CONFIG['gradient_clip'])
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

def val_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            total_loss += criterion(pred, y).item()

    return total_loss / len(loader)

def compute_metrics(model, loader, device):
    """Métricas estandarizadas para benchmark"""
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    # Métricas
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    # R²
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("✓ Funciones de entrenamiento definidas")


In [ ]:
def compute_metrics_hardware_mode(model, loader, device):
    """
    Calcula métricas FORZANDO el comportamiento de hardware (Ruido + Cuantización).
    Activa model.train() para encender el ruido, pero usa no_grad() para no entrenar.
    """
    # 1. Activamos modo train: Esto ENCIENDE el ruido gaussiano en AnalogLinear/Conv1d
    model.train() 
    
    y_true_all, y_pred_all = [], []

    # 2. Desactivamos gradientes: Solo queremos inferencia (predicción), no backprop
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    # 3. Cálculo de Métricas (Igual que la función estándar)
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("✓ Función de métricas Hardware-Mode definida")

import pandas as pd # Para mostrar la tabla bonita al final

print("="*80)
print(" COMPARATIVA FINAL: SOFTWARE (IDEAL) vs HARDWARE (REAL)")
print("="*80)

# Asumimos que 'models' es tu diccionario: {'LSTM': model_lstm, 'TCN': model_tcn, ...}
# Si solo tienes un modelo llamado 'model', crea el diccionario así:
# models = {'MiModelo': model} 

comparison_results = []

for name, model in models.items():
    print(f"\n🔹 Evaluando {name}...")
    
    # 1. Evaluación Ideal (Software / model.eval())
    # Sin ruido, solo cuantización si está implementada en eval, o float32 puro
    metrics_ideal = compute_metrics(model, test_loader, device)
    
    # 2. Evaluación Real (Hardware / model.train() + no_grad)
    # Con ruido gaussiano en los pesos y cuantización forzada
    metrics_hard = compute_metrics_hardware_mode(model, test_loader, device)

    # Mostrar en consola al vuelo
    print(f"   [Software] MSE: {metrics_ideal['mse']:.6f} | R²: {metrics_ideal['r2']:.6f}")
    print(f"   [Hardware] MSE: {metrics_hard['mse']:.6f}  | R²: {metrics_hard['r2']:.6f}")
    
    # Calcular degradación (Gap)
    mse_gap = metrics_hard['mse'] - metrics_ideal['mse']
    r2_drop = metrics_ideal['r2'] - metrics_hard['r2']
    print(f"   ⚠️ Degradación R²: -{r2_drop:.4f}")

    # Guardar para tabla final
    comparison_results.append({
        'Model': name,
        'Params': f"{sum(p.numel() for p in model.parameters()):,}",
        # Software
        'SW MSE': metrics_ideal['mse'],
        'SW R²': metrics_ideal['r2'],
        # Hardware
        'HW MSE': metrics_hard['mse'],
        'HW R²': metrics_hard['r2'],
        # Delta
        'Gap MSE': mse_gap,
        'Gap R²': r2_drop
    })

print("\n" + "="*80)
print(" RESUMEN FINAL DE IMPLEMENTACIÓN")
print("="*80)

df_results = pd.DataFrame(comparison_results)
# Formato bonito para visualizar en el notebook
print(df_results.to_string(index=False, float_format=lambda x: "{:.6f}".format(x)))

# Guardar a CSV por si acaso
df_results.to_csv('benchmark_hardware_comparison.csv', index=False)
print("\n✅ Benchmark completo guardado en 'benchmark_hardware_comparison.csv'")

In [ ]:
from torch.utils.data import DataLoader, random_split

# Reproducibilidad
torch.manual_seed(BENCHMARK_CONFIG['seed'])
np.random.seed(BENCHMARK_CONFIG['seed'])

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}\n")

# Crear dataset (data ya está cargado)
print("1️⃣ Creando dataset...")
dataset = UniversalFilterDataset(data)

# Split con seed fijo
print(f"\n2️⃣ Creando split (seed={BENCHMARK_CONFIG['seed']})...")
total = len(dataset)
train_size = int(BENCHMARK_CONFIG['train_ratio'] * total)
val_size = int(BENCHMARK_CONFIG['val_ratio'] * total)
test_size = total - train_size - val_size

trainds, valds, testds = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(BENCHMARK_CONFIG['seed'])
)

print(f"   Train: {len(trainds):,} | Val: {len(valds):,} | Test: {len(testds):,}")

# IMPORTANTE: Guardar indices del split para reproducibilidad
split_indices = {
    'train': trainds.indices,
    'val': valds.indices,
    'test': testds.indices
}
torch.save(split_indices, 'benchmark_split_indices.pt')
print(f"   ✓ Índices guardados en: benchmark_split_indices.pt")

# Data loaders
print(f"\n3️⃣ Creando data loaders...")
train_loader = DataLoader(
    trainds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=True,
    num_workers=BENCHMARK_CONFIG['num_workers'],
    pin_memory=BENCHMARK_CONFIG['pin_memory'],
    drop_last=BENCHMARK_CONFIG['drop_last']
)

val_loader = DataLoader(
    valds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

test_loader = DataLoader(
    testds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

print(f"   Batches - Train: {len(train_loader)} | Val: {len(val_loader)} | Test: {len(test_loader)}")


In [ ]:
import time

# # Modelo
# print(f"\n4️⃣ Inicializando modelo LSTM...")
# model = UniversalLSTM(
#     input_size=BENCHMARK_CONFIG['input_size'],
#     hidden=128,
#     layers=2,
#     dropout=0.2
# ).to(device)

# num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f"   Parámetros: {num_params:,}")

# -------------------- Modelo analogico --------------------
print(f"\n4️⃣ Inicializando modelo ANALÓGICO (Hardware-Aware)...")

# --- CAMBIO AQUÍ ---
# Instanciamos la clase AnalogLSTMModel que definimos antes
model = AnalogLSTMModel(
    input_size=BENCHMARK_CONFIG['input_size'],
    hidden_size=128,          # Equivalente a 'hidden' en tu clase anterior
    output_size=1,            # Tu UniversalLSTM tenía salida 1 hardcoded
    num_layers=2,             # Equivalente a 'layers'
    # Nuevos parámetros físicos:
    analog_levels=BENCHMARK_CONFIG['analog_levels'], 
    noise_std=BENCHMARK_CONFIG['noise_std']
).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Parámetros: {num_params:,}")
print(f"   Configuración Física: {BENCHMARK_CONFIG['analog_levels']} niveles, Ruido {BENCHMARK_CONFIG['noise_std']}")
# -------------------- Modelo analogico --------------------

# Loss & Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=BENCHMARK_CONFIG['learning_rate']
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=BENCHMARK_CONFIG['scheduler_patience']
)

# Training loop
print(f"\n5️⃣ Entrenando (epochs={BENCHMARK_CONFIG['epochs']})...\n")

best_val_loss = float('inf')
train_losses, val_losses = [], []
start_time = time.time()

for epoch in range(1, BENCHMARK_CONFIG['epochs'] + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = val_epoch(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    print(f"Epoch {epoch:2d} | Train: {train_loss:.6f} | Val: {val_loss:.6f}")

    # Guardar mejor modelo
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'config': BENCHMARK_CONFIG,
            'normalization': {
                'kmin': dataset.kmin.tolist(),
                'kmax': dataset.kmax.tolist()
            }
        }
        torch.save(checkpoint, 'lstm_best.pt')
        print(f"   💾 BEST SAVED (val_loss={val_loss:.6f})")

elapsed = time.time() - start_time
print(f"\n✓ Entrenamiento completado en {elapsed/60:.1f} minutos")


In [ ]:
# Evaluación final en test set
print(f"\n6️⃣ Evaluación final en test set...")
model.load_state_dict(torch.load('lstm_best.pt')['model_state_dict'])
test_metrics = compute_metrics(model, test_loader, device)

print(f"\n{'='*50}")
print(f"RESULTADOS LSTM BASELINE")
print(f"{'='*50}")
print(f"Best epoch: {best_epoch}")
print(f"Training time: {elapsed/60:.1f} min")
print(f"Parámetros: {num_params:,}")
print(f"\nTest Metrics:")
print(f"  MSE:  {test_metrics['mse']:.6f}")
print(f"  RMSE: {test_metrics['rmse']:.6f}")
print(f"  MAE:  {test_metrics['mae']:.6f}")
print(f"  R²:   {test_metrics['r2']:.6f}")
print(f"{'='*50}")

# Guardar resultados para benchmark
results = {
    'model_name': 'LSTM',
    'architecture': 'LSTM(128, layers=2)',
    'params': num_params,
    'train_time_min': elapsed/60,
    'best_epoch': best_epoch,
    'best_val_loss': best_val_loss,
    'test_metrics': test_metrics,
    'config': BENCHMARK_CONFIG,
    'train_losses': train_losses,
    'val_losses': val_losses
}

torch.save(results, 'lstm_benchmark_results.pt')
print(f"\n✓ Resultados guardados: lstm_benchmark_results.pt")


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss curves
axes[0].plot(train_losses, label='Train', linewidth=2)
axes[0].plot(val_losses, label='Val', linewidth=2)
axes[0].axvline(best_epoch-1, color='g', linestyle='--', label=f'Best (epoch {best_epoch})')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training History')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].set_yscale('log')

# Val/Train ratio
ratio = np.array(val_losses) / np.array(train_losses)
axes[1].plot(ratio, color='purple', linewidth=2)
axes[1].axhline(1, color='r', linestyle='--', label='No overfitting')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Val/Train Ratio')
axes[1].set_title('Overfitting Check')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('lstm_training_curves.png', dpi=150)
plt.show()

print("✓ Gráficas guardadas en: lstm_training_curves.png")


In [ ]:
# Descargar archivos importantes
from google.colab import files

print("Descargando archivos del benchmark...")

files.download('lstm_best.pt')
files.download('lstm_benchmark_results.pt')
files.download('benchmark_split_indices.pt')
files.download('lstm_training_curves.png')

print("\n✓ Descarga completa!")
print("\nArchivos descargados:")
print("  1. lstm_best.pt - Mejor modelo entrenado")
print("  2. lstm_benchmark_results.pt - Resultados y métricas")
print("  3. benchmark_split_indices.pt - Índices del split (CRUCIAL)")
print("  4. lstm_training_curves.png - Gráficas de entrenamiento")


In [ ]:
# Ejecuta esta celda y pégame el resultado:
results = torch.load('lstm_benchmark_results.pt')

print("="*50)
print("RESULTADOS FINALES LSTM")
print("="*50)
print(f"Modelo: {results['model_name']}")
print(f"Arquitectura: {results['architecture']}")
print(f"Parámetros: {results['params']:,}")
print(f"Tiempo entrenamiento: {results['train_time_min']:.1f} min")
print(f"Best epoch: {results['best_epoch']}")
print(f"Best val loss: {results['best_val_loss']:.6f}")
print(f"\nMétricas en TEST:")
print(f"  MSE:  {results['test_metrics']['mse']:.6f}")
print(f"  RMSE: {results['test_metrics']['rmse']:.6f}")
print(f"  MAE:  {results['test_metrics']['mae']:.6f}")
print(f"  R²:   {results['test_metrics']['r2']:.6f}")
print("="*50)


# ***TCN***

In [ ]:
import torch
import torch.nn as nn

# ============================================================
# TCN ADAPTADA PARA BENCHMARK (compatible con LSTM input)
# ============================================================

class FiLM(nn.Module):
    """Feature-wise Linear Modulation para control con knobs"""
    def __init__(self, channels, knob_dim):
        super().__init__()
        # Genera gamma (escala) y beta (desplazamiento) para cada canal
        self.gen = nn.Linear(knob_dim, channels * 2)

    def forward(self, x, knobs):
        # knobs: (batch, knob_dim)
        # x: (batch, channels, seq_len)
        params = self.gen(knobs).unsqueeze(2)  # (batch, channels*2, 1)
        gamma, beta = torch.chunk(params, 2, dim=1)
        return x * gamma + beta

class TemporalBlock(nn.Module):
    """Bloque TCN con dilated convolution + FiLM + residual"""
    def __init__(self, in_ch, out_ch, kernel_size, dilation, knob_dim):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation

        # Convolution con dilatación
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size,
                             padding=self.padding, dilation=dilation)

        # FiLM layer para modular con knobs
        self.film = FiLM(out_ch, knob_dim)

        # Activación y normalización
        self.act = nn.PReLU()
        self.norm = nn.GroupNorm(1, out_ch)

        # Residual connection
        self.res = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x, knobs):
        res = self.res(x)

        # Convolution con causal padding
        x = self.conv(x)
        if self.padding > 0:
            x = x[:, :, :-self.padding]  # Remover padding futuro

        # Modular con knobs via FiLM
        x = self.film(x, knobs)

        # Normalizar y aplicar activación
        x = self.norm(x)
        return self.act(x + res)

class UniversalTCN(nn.Module):
    """TCN para benchmark - Compatible con mismo input que LSTM"""
    def __init__(self, input_size=10, channels=128, num_layers=12, kernel_size=3):
        super().__init__()

        self.channels = channels
        self.num_layers = num_layers

        # Separar input en audio (1) y control (9 = 5 knobs + 4 topology)
        self.num_control = input_size - 1  # 9 señales de control

        # TCN layers con dilatación exponencial
        self.layers = nn.ModuleList()

        # Primera capa: 1 canal audio → channels
        self.layers.append(
            TemporalBlock(1, channels, kernel_size, dilation=1, knob_dim=self.num_control)
        )

        # Capas restantes con dilatación creciente
        for i in range(1, num_layers):
            dilation = 2 ** i
            self.layers.append(
                TemporalBlock(channels, channels, kernel_size, dilation, knob_dim=self.num_control)
            )

        # Output layer
        self.output = nn.Conv1d(channels, 1, kernel_size=1, bias=False)

    def forward(self, x):
        # x: (batch, seq_len, 10)
        # Separar audio y control
        audio = x[:, :, 0:1]  # (batch, seq_len, 1)
        control = x[:, :, 1:]  # (batch, seq_len, 9)

        # TCN espera (batch, channels, seq_len)
        audio = audio.permute(0, 2, 1)  # (batch, 1, seq_len)

        # Control toma el primer timestep (es constante en toda la secuencia)
        knobs = control[:, 0, :]  # (batch, 9)

        # Procesar con TCN
        out = audio
        for layer in self.layers:
            out = layer(out, knobs)

        # Output layer
        out = self.output(out)  # (batch, 1, seq_len)

        # Volver a formato (batch, seq_len, 1)
        out = out.permute(0, 2, 1)

        return out

print("✓ Modelo TCN definido (compatible con benchmark)")

# --- Bloques Auxiliares Analógicos para TCN ---

class AnalogFiLM(nn.Module):
    def __init__(self, channels, knob_dim, analog_levels=32, noise_std=0.02):
        super().__init__()
        # Usamos AnalogLinear para generar los coeficientes
        self.gen = AnalogLinear(knob_dim, channels * 2, 
                                analog_levels=analog_levels, noise_std=noise_std)

    def forward(self, x, knobs):
        params = self.gen(knobs).unsqueeze(2)
        gamma, beta = torch.chunk(params, 2, dim=1)
        return x * gamma + beta

class AnalogTemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, knob_dim, 
                 analog_levels=32, noise_std=0.02):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation

        # CONVOLUCIÓN ANALÓGICA
        self.conv = AnalogConv1d(in_ch, out_ch, kernel_size,
                                 padding=self.padding, dilation=dilation,
                                 analog_levels=analog_levels, noise_std=noise_std)

        # FiLM ANALÓGICO
        self.film = AnalogFiLM(out_ch, knob_dim, 
                               analog_levels=analog_levels, noise_std=noise_std)

        self.act = nn.PReLU()
        self.norm = nn.GroupNorm(1, out_ch)

        # RESIDUAL ANALÓGICA (Si es necesaria proyección 1x1)
        if in_ch != out_ch:
            self.res = AnalogConv1d(in_ch, out_ch, 1, 
                                    analog_levels=analog_levels, noise_std=noise_std)
        else:
            self.res = nn.Identity()

    def forward(self, x, knobs):
        res = self.res(x)
        x = self.conv(x)
        if self.padding > 0:
            x = x[:, :, :-self.padding]
        
        x = self.film(x, knobs)
        x = self.norm(x)
        return self.act(x + res)

# --- Modelo Principal TCN Analógico ---

class AnalogTCNModel(nn.Module):
    def __init__(self, input_size=10, channels=128, num_layers=12, kernel_size=3,
                 analog_levels=32, noise_std=0.02):
        super().__init__()

        self.channels = channels
        self.num_layers = num_layers
        self.num_control = input_size - 1

        self.layers = nn.ModuleList()

        # Primera capa (Input -> Channels)
        self.layers.append(
            AnalogTemporalBlock(1, channels, kernel_size, dilation=1, 
                                knob_dim=self.num_control,
                                analog_levels=analog_levels, noise_std=noise_std)
        )

        # Capas profundas
        for i in range(1, num_layers):
            dilation = 2 ** i
            self.layers.append(
                AnalogTemporalBlock(channels, channels, kernel_size, dilation, 
                                    knob_dim=self.num_control,
                                    analog_levels=analog_levels, noise_std=noise_std)
            )

        # Output Layer (1x1 Conv Analógica)
        self.output = AnalogConv1d(channels, 1, kernel_size=1, bias=False,
                                   analog_levels=analog_levels, noise_std=noise_std)

    def forward(self, x):
        audio = x[:, :, 0:1]
        control = x[:, :, 1:]
        
        audio = audio.permute(0, 2, 1) # (B, 1, T)
        knobs = control[:, 0, :]       # (B, 9)

        out = audio
        for layer in self.layers:
            out = layer(out, knobs)

        out = self.output(out)
        out = out.permute(0, 2, 1) # (B, T, 1)

        return out

print("✓ Modelo AnalogTCN definido")


In [ ]:
from torch.utils.data import DataLoader, Subset
import numpy as np

# Reproducibilidad
torch.manual_seed(BENCHMARK_CONFIG['seed'])
np.random.seed(BENCHMARK_CONFIG['seed'])

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}\n")

# Crear dataset (data ya está cargado)
print("1️⃣ Creando dataset...")
dataset = UniversalFilterDataset(data)

# ============================================================
# CARGAR MISMO SPLIT QUE LSTM (CRÍTICO PARA BENCHMARK)
# ============================================================
print(f"\n2️⃣ Cargando MISMO split que LSTM...")

# Cargar índices del entrenamiento LSTM
split_indices = torch.load('benchmark_split_indices.pt')

print(f"   ⚠️ IMPORTANTE: Usando EXACTAMENTE los mismos datos que LSTM")
print(f"   Train indices: {len(split_indices['train'])} muestras")
print(f"   Val indices: {len(split_indices['val'])} muestras")
print(f"   Test indices: {len(split_indices['test'])} muestras")

# Crear subsets con los MISMOS índices
trainds = Subset(dataset, split_indices['train'])
valds = Subset(dataset, split_indices['val'])
testds = Subset(dataset, split_indices['test'])

print(f"   Train: {len(trainds):,} | Val: {len(valds):,} | Test: {len(testds):,}")
print(f"   ✅ Split idéntico al LSTM verificado")

# Data loaders (MISMO batch size, etc.)
print(f"\n3️⃣ Creando data loaders...")
train_loader = DataLoader(
    trainds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=True,  # Shuffle está bien, los índices ya definen el split
    num_workers=BENCHMARK_CONFIG['num_workers'],
    pin_memory=BENCHMARK_CONFIG['pin_memory'],
    drop_last=BENCHMARK_CONFIG['drop_last']
)

val_loader = DataLoader(
    valds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

test_loader = DataLoader(
    testds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

print(f"   Batches - Train: {len(train_loader)} | Val: {len(val_loader)} | Test: {len(test_loader)}")
print(f"   ✅ Data loaders listos con configuración idéntica a LSTM")


In [ ]:
import time

# # Modelo TCN
# print(f"\n4️⃣ Inicializando modelo TCN...")
# model = UniversalTCN(
#     input_size=BENCHMARK_CONFIG['input_size'],
#     channels=128,
#     num_layers=12,
#     kernel_size=3
# ).to(device)

# num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(f"   Parámetros: {num_params:,}")

# -------------------- Modelo analogico --------------------
# Modelo TCN Analógico
print(f"\n4️⃣ Inicializando modelo TCN (Hardware-Aware)...")

model = AnalogTCNModel(
    input_size=BENCHMARK_CONFIG['input_size'],
    channels=128,       # Mismos canales que tu benchmark original
    num_layers=12,      # Mismas capas
    kernel_size=3,      # Mismo kernel
    # --- Parámetros Físicos ---
    analog_levels=BENCHMARK_CONFIG['analog_levels'],
    noise_std=BENCHMARK_CONFIG['noise_std']
).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Parámetros: {num_params:,}")
print(f"   Modo Físico: {BENCHMARK_CONFIG['analog_levels']} niveles | Ruido {BENCHMARK_CONFIG['noise_std']}")
# -------------------- Modelo analogico --------------------


# Loss & Optimizer (MISMO que LSTM)
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=BENCHMARK_CONFIG['learning_rate']
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=BENCHMARK_CONFIG['scheduler_patience']
)

# Training loop
print(f"\n5️⃣ Entrenando TCN (epochs={BENCHMARK_CONFIG['epochs']})...\n")

best_val_loss = float('inf')
train_losses, val_losses = [], []
start_time = time.time()

for epoch in range(1, BENCHMARK_CONFIG['epochs'] + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = val_epoch(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    print(f"Epoch {epoch:2d} | Train: {train_loss:.6f} | Val: {val_loss:.6f}")

    # Guardar mejor modelo
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'config': BENCHMARK_CONFIG,
            'normalization': {
                'kmin': dataset.kmin.tolist(),
                'kmax': dataset.kmax.tolist()
            },
            'split_indices': split_indices  # Guardar para verificación
        }
        torch.save(checkpoint, 'tcn_best.pt')
        print(f"   💾 BEST SAVED (val_loss={val_loss:.6f})")

elapsed = time.time() - start_time
print(f"\n✓ Entrenamiento completado en {elapsed/60:.1f} minutos")


# ***RNN***

In [ ]:
import torch.nn as nn

# ============================================================
# RNN ADAPTADA PARA BENCHMARK (compatible con LSTM/TCN input)
# ============================================================

class UniversalRNN(nn.Module):
    """
    RNN con Context MLP para benchmark.
    Compatible con mismo input que LSTM/TCN.
    """
    def __init__(self, input_size=10, hidden_size=256, num_layers=3):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Context dimension: 9 (5 knobs + 4 topology)
        self.context_dim = input_size - 1  # 10 - 1 = 9

        # A. Context MLP (Cerebro Lógico)
        # Procesa knobs + topology antes de la RNN
        self.context_mlp = nn.Sequential(
            nn.Linear(self.context_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.Tanh()
        )

        # B. RNN Vanilla (Cerebro Temporal)
        # Input: audio(1) + context_embedding(32) = 33
        self.rnn = nn.RNN(
            input_size=1 + 32,  # Audio + contexto procesado
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            nonlinearity='tanh',
            dropout=0.2 if num_layers > 1 else 0
        )

        # C. Output Head
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 128),
            nn.GELU(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        # x: (batch, seq_len, 10)

        # Separar audio y contexto
        audio = x[:, :, 0:1]          # (batch, seq_len, 1)
        raw_context = x[:, 0, 1:]     # (batch, 9) - tomar tiempo 0 (es constante)

        # Procesar contexto con MLP
        ctx_emb = self.context_mlp(raw_context)  # (batch, 32)

        # Expandir contexto para toda la secuencia
        seq_len = audio.size(1)
        ctx_emb_expanded = ctx_emb.unsqueeze(1).repeat(1, seq_len, 1)  # (batch, seq_len, 32)

        # Concatenar audio + contexto procesado
        rnn_input = torch.cat([audio, ctx_emb_expanded], dim=2)  # (batch, seq_len, 33)

        # RNN forward
        rnn_out, _ = self.rnn(rnn_input)  # (batch, seq_len, hidden_size)

        # Output head
        output = self.head(rnn_out)  # (batch, seq_len, 1)

        return output

print("✅ Modelo RNN definido (compatible con benchmark)")

class AnalogRNNModel(nn.Module):
    """
    Versión Hardware-Aware de la UniversalRNN.
    """
    def __init__(self, input_size=10, hidden_size=256, num_layers=3, 
                 analog_levels=32, noise_std=0.02):
        super(AnalogRNNModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.context_dim = input_size - 1 

        # A. Context MLP Analógico
        # Usamos AnalogLinear porque esta lógica también corre en el chip
        self.context_mlp = nn.Sequential(
            AnalogLinear(self.context_dim, 64, analog_levels=analog_levels, noise_std=noise_std),
            nn.ReLU(),
            AnalogLinear(64, 32, analog_levels=analog_levels, noise_std=noise_std),
            nn.Tanh()
        )

        # B. RNN Vanilla
        # Al igual que con LSTM, usaremos nn.RNN estándar e inyectaremos ruido manualmente
        self.rnn = nn.RNN(
            input_size=1 + 32, 
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            nonlinearity='tanh',
            dropout=0.2 if num_layers > 1 else 0
        )

        # C. Output Head Analógico
        self.head = nn.Sequential(
            AnalogLinear(hidden_size, 128, analog_levels=analog_levels, noise_std=noise_std),
            nn.GELU(),
            AnalogLinear(128, 1, analog_levels=analog_levels, noise_std=noise_std)
        )

    def forward(self, x):
        # --- INYECCIÓN DE RUIDO EN RNN ---
        if self.training:
            with torch.no_grad():
                for name, param in self.rnn.named_parameters():
                    if 'weight' in name:
                        noise = torch.randn_like(param) * 0.01 
                        param.add_(noise)

        # --- Lógica estándar ---
        audio = x[:, :, 0:1]         
        raw_context = x[:, 0, 1:]    

        ctx_emb = self.context_mlp(raw_context)
        seq_len = audio.size(1)
        ctx_emb_expanded = ctx_emb.unsqueeze(1).repeat(1, seq_len, 1)
        rnn_input = torch.cat([audio, ctx_emb_expanded], dim=2)

        rnn_out, _ = self.rnn(rnn_input)
        output = self.head(rnn_out)

        # --- LIMPIEZA DE RUIDO RNN ---
        # (Nota: PyTorch acumula gradientes en la versión ruidosa, lo cual es correcto para QAT)
        # Aquí no revertimos la resta para ahorrar cómputo, ya que el ruido es aleatorio centrado en 0
        
        return output

print("✓ Modelo AnalogRNN definido")

In [ ]:
import torch.optim as optim
import numpy as np

def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            model.parameters(),
            BENCHMARK_CONFIG['gradient_clip']
        )
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

def val_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            total_loss += criterion(pred, y).item()

    return total_loss / len(loader)

def compute_metrics(model, loader, device):
    """Métricas estandarizadas para benchmark"""
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    # Métricas
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    # R²
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("✅ Funciones de entrenamiento definidas")


In [ ]:
from torch.utils.data import DataLoader, Subset
import numpy as np
import os

# Reproducibilidad
torch.manual_seed(BENCHMARK_CONFIG['seed'])
np.random.seed(BENCHMARK_CONFIG['seed'])

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 1️⃣ Crear dataset
print(f"\n1️⃣ Creando dataset...")
dataset = UniversalFilterDataset(data)

# 2️⃣ CARGAR MISMO SPLIT (desde Drive)
print(f"\n2️⃣ Cargando split desde Drive...")
SPLIT_PATH = 'benchmark_split_indices.pt'

if not os.path.exists(SPLIT_PATH):
    print("❌ ERROR: benchmark_split_indices.pt no encontrado en Drive")
    print("   Debes subirlo primero!")
    raise FileNotFoundError(f"No se encuentra: {SPLIT_PATH}")
else:
    split_indices = torch.load(SPLIT_PATH)
    print(f"✅ Split cargado desde Drive (mentira lo subí a collab)")
    print(f"   Train indices: {len(split_indices['train']):,} muestras")
    print(f"   Val indices: {len(split_indices['val']):,} muestras")
    print(f"   Test indices: {len(split_indices['test']):,} muestras")

    # Crear subsets con los MISMOS índices que LSTM/TCN
    trainds = Subset(dataset, split_indices['train'])
    valds = Subset(dataset, split_indices['val'])
    testds = Subset(dataset, split_indices['test'])

    print(f"   Train: {len(trainds):,} | Val: {len(valds):,} | Test: {len(testds):,}")
    print(f"   ✓ Split idéntico al LSTM/TCN")

# 3️⃣ Data loaders
print(f"\n3️⃣ Creando data loaders...")
train_loader = DataLoader(
    trainds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=True,
    num_workers=BENCHMARK_CONFIG['num_workers'],
    pin_memory=BENCHMARK_CONFIG['pin_memory'],
    drop_last=BENCHMARK_CONFIG['drop_last']
)

val_loader = DataLoader(
    valds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

test_loader = DataLoader(
    testds,
    batch_size=BENCHMARK_CONFIG['batch_size'],
    shuffle=False,
    num_workers=BENCHMARK_CONFIG['num_workers']
)

print(f"   Batches - Train: {len(train_loader)} | Val: {len(val_loader)} | Test: {len(test_loader)}")
print(f"\n✅ Data preparada con configuración idéntica a LSTM/TCN")


In [ ]:
import time

# Modelo RNN
print(f"\n4️⃣ Inicializando modelo RNN...")
model = UniversalRNN(
    input_size=BENCHMARK_CONFIG['input_size'],
    hidden_size=256,
    num_layers=3
).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Parámetros: {num_params:,}")

# ------------------------- Modelo Analogico ----------------------------
print(f"\n4️⃣ Inicializando modelo RNN (Hardware-Aware)...")

model = AnalogRNNModel(
    input_size=BENCHMARK_CONFIG['input_size'],
    hidden_size=256,    # Mismo hidden size que tu benchmark original
    num_layers=3,       # Mismas capas
    # --- Parámetros Físicos ---
    analog_levels=BENCHMARK_CONFIG['analog_levels'],
    noise_std=BENCHMARK_CONFIG['noise_std']
).to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Parámetros: {num_params:,}")
print(f"   Modo Físico: {BENCHMARK_CONFIG['analog_levels']} niveles | Ruido {BENCHMARK_CONFIG['noise_std']}")
# ------------------------- Modelo Analogico ----------------------------

# Loss & Optimizer (MISMO que LSTM/TCN)
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr=BENCHMARK_CONFIG['learning_rate']
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    patience=BENCHMARK_CONFIG['scheduler_patience']
)

# Training loop
print(f"\n5️⃣ Entrenando RNN (epochs={BENCHMARK_CONFIG['epochs']})...\n")

best_val_loss = float('inf')
train_losses, val_losses = [], []
start_time = time.time()

for epoch in range(1, BENCHMARK_CONFIG['epochs'] + 1):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = val_epoch(model, val_loader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    print(f"Epoch {epoch:2d} | Train: {train_loss:.6f} | Val: {val_loss:.6f}")

    # Guardar mejor modelo
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch

        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'config': BENCHMARK_CONFIG,
            'normalization': {
                'kmin': dataset.kmin.tolist(),
                'kmax': dataset.kmax.tolist()
            },
            'split_indices': split_indices
        }
        torch.save(checkpoint, 'rnn_best.pt')
        print(f"   💾 BEST SAVED (val_loss={val_loss:.6f})")

elapsed = time.time() - start_time
print(f"\n✅ Entrenamiento completado en {elapsed/60:.1f} minutos")


In [ ]:
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# ============================================================
# VER ARCHIVOS EN COLAB
# ============================================================

print("\nContenido de /content/:")
!ls -lh /content/

print("\n" + "="*60)
print("Buscando archivos .pt en /content/:")
!find /content/ -name "*.pt" -type f 2>/dev/null
print("="*60)


In [ ]:
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Rutas en /content/
paths = {
    'benchmark_split_indices': '/content/benchmark_split_indices.pt',
    'lstm': '/content/lstm_best.pt',
    'tcn': '/content/tcn_best.pt',
    'rnn': '/content/rnn_best.pt'
}

# Verificar
print("\n✅ Archivos encontrados:")
for name, path in paths.items():
    size = os.path.getsize(path) / (1024*1024)
    print(f"   {name:30s} | {size:.2f} MB")

print("\n✅ Listo para cargar modelos")


In [ ]:
import torch.nn as nn

# 1. LSTM
class UniversalLSTM(nn.Module):
    def __init__(self, input_size=10, hidden=128, layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, layers, batch_first=True,
                           dropout=dropout if layers > 1 else 0)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out)

# 2. TCN
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super().__init__()
        self.chomp_size = chomp_size
    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class UniversalTCN(nn.Module):
    def __init__(self, input_size=10, num_channels=[64, 128, 256], kernel_size=3, dropout=0.2):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = input_size if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1,
                                    dilation=dilation_size, padding=(kernel_size-1) * dilation_size,
                                    dropout=dropout)]
        self.network = nn.Sequential(*layers)
        self.fc = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        x = x.transpose(1, 2)
        y = self.network(x)
        y = y.transpose(1, 2)
        return self.fc(y)

# 3. RNN
class UniversalRNN(nn.Module):
    def __init__(self, input_size=10, hidden_size=256, num_layers=3):
        super().__init__()
        self.context_dim = input_size - 1
        self.context_mlp = nn.Sequential(
            nn.Linear(self.context_dim, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.Tanh()
        )
        self.rnn = nn.RNN(input_size=1+32, hidden_size=hidden_size, num_layers=num_layers,
                         batch_first=True, nonlinearity='tanh', dropout=0.2 if num_layers > 1 else 0)
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 128), nn.GELU(), nn.Linear(128, 1)
        )

    def forward(self, x):
        audio = x[:, :, 0:1]
        raw_context = x[:, 0, 1:]
        ctx_emb = self.context_mlp(raw_context)
        seq_len = audio.size(1)
        ctx_emb_expanded = ctx_emb.unsqueeze(1).repeat(1, seq_len, 1)
        rnn_input = torch.cat([audio, ctx_emb_expanded], dim=2)
        rnn_out, _ = self.rnn(rnn_input)
        return self.head(rnn_out)

print("✅ Arquitecturas definidas")


In [ ]:
# Ver qué arquitectura tiene el TCN guardado
tcn_checkpoint = torch.load(paths['tcn'], map_location='cpu')

print("Claves en el checkpoint:")
print(tcn_checkpoint.keys())

print("\n" + "="*60)
print("Arquitectura guardada (primeras capas):")
for i, (key, value) in enumerate(tcn_checkpoint['model_state_dict'].items()):
    if i < 20:  # Primeras 20 capas
        print(f"{key:50s} | Shape: {value.shape}")
    else:
        break

# Ver si hay config guardada
if 'config' in tcn_checkpoint:
    print("\n" + "="*60)
    print("Config guardada:")
    print(tcn_checkpoint['config'])


In [ ]:
import torch.nn as nn

# ============================================================
# ARQUITECTURAS COMPLETAS
# ============================================================

# 1. LSTM (igual que antes)
class UniversalLSTM(nn.Module):
    def __init__(self, input_size=10, hidden=128, layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, layers, batch_first=True,
                           dropout=dropout if layers > 1 else 0)
        self.fc = nn.Linear(hidden, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out)

# 2. TCN con FiLM (arquitectura real del checkpoint)
class FiLM(nn.Module):
    def __init__(self, channels, knob_dim):
        super().__init__()
        self.gen = nn.Linear(knob_dim, channels * 2)

    def forward(self, x, knobs):
        params = self.gen(knobs).unsqueeze(2)
        gamma, beta = torch.chunk(params, 2, dim=1)
        return x * gamma + beta

class TCNLayer(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation, knob_dim):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_ch, out_ch, kernel_size, padding=self.padding, dilation=dilation)
        self.film = FiLM(out_ch, knob_dim)
        self.act = nn.PReLU()
        self.norm = nn.GroupNorm(1, out_ch)
        self.res = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()

    def forward(self, x, knobs):
        res = self.res(x)
        x = self.conv(x)
        if self.padding > 0:
            x = x[:, :, :-self.padding]
        x = self.film(x, knobs)
        x = self.norm(x)
        return self.act(x + res)

class UniversalTCN(nn.Module):
    """TCN con FiLM - Adaptado para benchmark (input de shape batch, seq, 10)"""
    def __init__(self, input_size=10, num_knobs=9):
        super().__init__()
        self.channels = 128
        self.num_layers = 12  # 12 capas con dilaciones
        self.layers = nn.ModuleList()

        # Primera capa: 1 canal de audio -> 128 canales
        self.layers.append(TCNLayer(1, self.channels, 3, 1, num_knobs))

        # Capas restantes con dilaciones crecientes
        for i in range(1, self.num_layers):
            dilation = 2 ** i
            self.layers.append(TCNLayer(self.channels, self.channels, 3, dilation, num_knobs))

        # Capa de salida
        self.output = nn.Conv1d(self.channels, 1, kernel_size=1, bias=False)

    def forward(self, x):
        # x: (batch, seq_len, 10)
        # Separar audio (columna 0) y knobs (columnas 1-9)
        audio = x[:, :, 0:1]  # (batch, seq_len, 1)
        knobs = x[:, 0, 1:]   # (batch, 9) - tomar primer timestep

        # TCN espera (batch, channels, time)
        audio = audio.transpose(1, 2)  # (batch, 1, seq_len)

        # Forward a través de las capas
        y = audio
        for layer in self.layers:
            y = layer(y, knobs)

        # Salida
        y = self.output(y)  # (batch, 1, seq_len)

        # Volver a (batch, seq_len, 1)
        y = y.transpose(1, 2)
        return y

# 3. RNN (igual que antes)
class UniversalRNN(nn.Module):
    def __init__(self, input_size=10, hidden_size=256, num_layers=3):
        super().__init__()
        self.context_dim = input_size - 1
        self.context_mlp = nn.Sequential(
            nn.Linear(self.context_dim, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.Tanh()
        )
        self.rnn = nn.RNN(input_size=1+32, hidden_size=hidden_size, num_layers=num_layers,
                         batch_first=True, nonlinearity='tanh', dropout=0.2 if num_layers > 1 else 0)
        self.head = nn.Sequential(
            nn.Linear(hidden_size, 128), nn.GELU(), nn.Linear(128, 1)
        )

    def forward(self, x):
        audio = x[:, :, 0:1]
        raw_context = x[:, 0, 1:]
        ctx_emb = self.context_mlp(raw_context)
        seq_len = audio.size(1)
        ctx_emb_expanded = ctx_emb.unsqueeze(1).repeat(1, seq_len, 1)
        rnn_input = torch.cat([audio, ctx_emb_expanded], dim=2)
        rnn_out, _ = self.rnn(rnn_input)
        return self.head(rnn_out)

print("✅ Arquitecturas definidas (LSTM, TCN con FiLM, RNN)")


In [ ]:
from torch.utils.data import DataLoader, Subset

# Cargar split
print("Cargando split...")
split_indices = torch.load(paths['benchmark_split_indices'])
print(f"✅ Split cargado:")
print(f"   Train: {len(split_indices['train']):,}")
print(f"   Val:   {len(split_indices['val']):,}")
print(f"   Test:  {len(split_indices['test']):,}")

# Crear test loader (asume que ya tienes 'dataset' creado de antes)
testds = Subset(dataset, split_indices['test'])
test_loader = DataLoader(testds, batch_size=32, shuffle=False, num_workers=2)

print(f"\n✅ Test loader listo | {len(test_loader)} batches")


In [ ]:
import numpy as np
import pandas as pd

def compute_metrics(model, loader, device):
    """Calcula métricas en un DataLoader"""
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("="*70)
print("EVALUANDO MODELOS EN TEST SET...")
print("="*70)

results = []

for name, model in models.items():
    print(f"\n{name}:")
    metrics = compute_metrics(model, test_loader, device)

    results.append({
        'Model': name,
        'Params': f"{sum(p.numel() for p in model.parameters()):,}",
        'Test MSE': f"{metrics['mse']:.8f}",
        'Test RMSE': f"{metrics['rmse']:.6f}",
        'Test MAE': f"{metrics['mae']:.6f}",
        'Test R²': f"{metrics['r2']:.6f}"
    })

    for k, v in metrics.items():
        print(f"  {k.upper():6s}: {v:.8f}")

print("\n✅ Evaluación completada")


In [ ]:
def compute_metrics_hardware_mode(model, loader, device):
    """
    Calcula métricas FORZANDO el comportamiento de hardware (Ruido + Cuantización).
    Activa model.train() para encender el ruido, pero usa no_grad() para no entrenar.
    """
    # 1. Activamos modo train: Esto ENCIENDE el ruido gaussiano en AnalogLinear/Conv1d
    model.train() 
    
    y_true_all, y_pred_all = [], []

    # 2. Desactivamos gradientes: Solo queremos inferencia (predicción), no backprop
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    # 3. Cálculo de Métricas (Igual que la función estándar)
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("✓ Función de métricas Hardware-Mode definida")

import pandas as pd # Para mostrar la tabla bonita al final

print("="*80)
print(" COMPARATIVA FINAL: SOFTWARE (IDEAL) vs HARDWARE (REAL)")
print("="*80)

# Asumimos que 'models' es tu diccionario: {'LSTM': model_lstm, 'TCN': model_tcn, ...}
# Si solo tienes un modelo llamado 'model', crea el diccionario así:
# models = {'MiModelo': model} 

comparison_results = []

for name, model in models.items():
    print(f"\n🔹 Evaluando {name}...")
    
    # 1. Evaluación Ideal (Software / model.eval())
    # Sin ruido, solo cuantización si está implementada en eval, o float32 puro
    metrics_ideal = compute_metrics(model, test_loader, device)
    
    # 2. Evaluación Real (Hardware / model.train() + no_grad)
    # Con ruido gaussiano en los pesos y cuantización forzada
    metrics_hard = compute_metrics_hardware_mode(model, test_loader, device)

    # Mostrar en consola al vuelo
    print(f"   [Software] MSE: {metrics_ideal['mse']:.6f} | R²: {metrics_ideal['r2']:.6f}")
    print(f"   [Hardware] MSE: {metrics_hard['mse']:.6f}  | R²: {metrics_hard['r2']:.6f}")
    
    # Calcular degradación (Gap)
    mse_gap = metrics_hard['mse'] - metrics_ideal['mse']
    r2_drop = metrics_ideal['r2'] - metrics_hard['r2']
    print(f"   ⚠️ Degradación R²: -{r2_drop:.4f}")

    # Guardar para tabla final
    comparison_results.append({
        'Model': name,
        'Params': f"{sum(p.numel() for p in model.parameters()):,}",
        # Software
        'SW MSE': metrics_ideal['mse'],
        'SW R²': metrics_ideal['r2'],
        # Hardware
        'HW MSE': metrics_hard['mse'],
        'HW R²': metrics_hard['r2'],
        # Delta
        'Gap MSE': mse_gap,
        'Gap R²': r2_drop
    })

print("\n" + "="*80)
print(" RESUMEN FINAL DE IMPLEMENTACIÓN")
print("="*80)

df_results = pd.DataFrame(comparison_results)
# Formato bonito para visualizar en el notebook
print(df_results.to_string(index=False, float_format=lambda x: "{:.6f}".format(x)))

# Guardar a CSV por si acaso
df_results.to_csv('benchmark_hardware_comparison.csv', index=False)
print("\n✅ Benchmark completo guardado en 'benchmark_hardware_comparison.csv'")

✓ Función de métricas Hardware-Mode definida


In [ ]:
df = pd.DataFrame(results)

# Ordenar por R² (mejor primero)
df_sorted = df.copy()
df_sorted['R2_float'] = df_sorted['Test R²'].astype(float)
df_sorted = df_sorted.sort_values('R2_float', ascending=False)
df_sorted = df_sorted.drop('R2_float', axis=1)

print("\n" + "="*80)
print("COMPARACIÓN FINAL DE MODELOS - TEST SET")
print("="*80)
print(df_sorted.to_string(index=False))
print("="*80)
print("\nNOTAS:")
print("- Todos evaluados en el MISMO test set (15% del dataset, 14,907 muestras)")
print("- Test set NUNCA visto durante el entrenamiento")
print("- Split idéntico cargado desde: benchmark_split_indices.pt")
print("- Métricas comparables directamente")
print("="*80)

# Guardar resultados
comparison_results = {
    'results': results,
    'test_set_size': len(split_indices['test']),
    'models_compared': list(models.keys())
}
torch.save(comparison_results, 'benchmark_comparison.pt')
print("\n✅ Resultados guardados en: benchmark_comparison.pt")


In [ ]:
print("Modelos en memoria:")
print(f"  Modelos cargados: {list(models.keys())}")
print(f"  Total: {len(models)} modelo(s)")

for name in models.keys():
    num_params = sum(p.numel() for p in models[name].parameters())
    print(f"  - {name}: {num_params:,} parámetros")


In [ ]:
models = {}

# 1. LSTM
try:
    print("Cargando LSTM...")
    lstm_model = UniversalLSTM(input_size=10, hidden=128, layers=2, dropout=0.2).to(device)
    lstm_checkpoint = torch.load(paths['lstm'], map_location=device)
    lstm_model.load_state_dict(lstm_checkpoint['model_state_dict'])
    lstm_model.eval()
    models['LSTM'] = lstm_model
    num_params_lstm = sum(p.numel() for p in lstm_model.parameters())
    print(f"✅ LSTM cargado | Params: {num_params_lstm:,}")
except Exception as e:
    print(f"❌ Error cargando LSTM: {e}")

# 2. TCN
try:
    print("\nCargando TCN...")
    tcn_model = UniversalTCN(input_size=10, num_knobs=9).to(device)
    tcn_checkpoint = torch.load(paths['tcn'], map_location=device)
    tcn_model.load_state_dict(tcn_checkpoint['model_state_dict'])
    tcn_model.eval()
    models['TCN'] = tcn_model
    num_params_tcn = sum(p.numel() for p in tcn_model.parameters())
    print(f"✅ TCN cargado  | Params: {num_params_tcn:,}")
except Exception as e:
    print(f"❌ Error cargando TCN: {e}")

# 3. RNN
try:
    print("\nCargando RNN...")
    rnn_model = UniversalRNN(input_size=10, hidden_size=256, num_layers=3).to(device)
    rnn_checkpoint = torch.load(paths['rnn'], map_location=device)
    rnn_model.load_state_dict(rnn_checkpoint['model_state_dict'])
    rnn_model.eval()
    models['RNN'] = rnn_model
    num_params_rnn = sum(p.numel() for p in rnn_model.parameters())
    print(f"✅ RNN cargado  | Params: {num_params_rnn:,}")
except Exception as e:
    print(f"❌ Error cargando RNN: {e}")

print(f"\n{'='*60}")
print(f"✅ {len(models)} modelo(s) cargado(s): {list(models.keys())}")
print(f"{'='*60}")


In [ ]:
import numpy as np
import pandas as pd

def compute_metrics(model, loader, device):
    """Calcula métricas en un DataLoader"""
    model.eval()
    y_true_all, y_pred_all = [], []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("="*70)
print("EVALUANDO 3 MODELOS EN TEST SET...")
print("="*70)

results = []

for name, model in models.items():
    print(f"\n{name}:")
    metrics = compute_metrics(model, test_loader, device)

    results.append({
        'Model': name,
        'Params': f"{sum(p.numel() for p in model.parameters()):,}",
        'Test MSE': f"{metrics['mse']:.8f}",
        'Test RMSE': f"{metrics['rmse']:.6f}",
        'Test MAE': f"{metrics['mae']:.6f}",
        'Test R²': f"{metrics['r2']:.6f}"
    })

    for k, v in metrics.items():
        print(f"  {k.upper():6s}: {v:.8f}")

print("\n" + "="*70)
print("✅ Evaluación completada - 3 modelos")
print("="*70)


In [ ]:
def compute_metrics_hardware_mode(model, loader, device):
    """
    Calcula métricas FORZANDO el comportamiento de hardware (Ruido + Cuantización).
    Activa model.train() para encender el ruido, pero usa no_grad() para no entrenar.
    """
    # 1. Activamos modo train: Esto ENCIENDE el ruido gaussiano en AnalogLinear/Conv1d
    model.train() 
    
    y_true_all, y_pred_all = [], []

    # 2. Desactivamos gradientes: Solo queremos inferencia (predicción), no backprop
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            y_true_all.append(y.cpu().numpy())
            y_pred_all.append(pred.cpu().numpy())

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)

    # 3. Cálculo de Métricas (Igual que la función estándar)
    mse = np.mean((y_true - y_pred)**2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_true - y_pred))

    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    r2 = 1 - (ss_res / ss_tot)

    return {
        'mse': float(mse),
        'rmse': float(rmse),
        'mae': float(mae),
        'r2': float(r2)
    }

print("✓ Función de métricas Hardware-Mode definida")

import pandas as pd # Para mostrar la tabla bonita al final

print("="*80)
print(" COMPARATIVA FINAL: SOFTWARE (IDEAL) vs HARDWARE (REAL)")
print("="*80)

# Asumimos que 'models' es tu diccionario: {'LSTM': model_lstm, 'TCN': model_tcn, ...}
# Si solo tienes un modelo llamado 'model', crea el diccionario así:
# models = {'MiModelo': model} 

comparison_results = []

for name, model in models.items():
    print(f"\n🔹 Evaluando {name}...")
    
    # 1. Evaluación Ideal (Software / model.eval())
    # Sin ruido, solo cuantización si está implementada en eval, o float32 puro
    metrics_ideal = compute_metrics(model, test_loader, device)
    
    # 2. Evaluación Real (Hardware / model.train() + no_grad)
    # Con ruido gaussiano en los pesos y cuantización forzada
    metrics_hard = compute_metrics_hardware_mode(model, test_loader, device)

    # Mostrar en consola al vuelo
    print(f"   [Software] MSE: {metrics_ideal['mse']:.6f} | R²: {metrics_ideal['r2']:.6f}")
    print(f"   [Hardware] MSE: {metrics_hard['mse']:.6f}  | R²: {metrics_hard['r2']:.6f}")
    
    # Calcular degradación (Gap)
    mse_gap = metrics_hard['mse'] - metrics_ideal['mse']
    r2_drop = metrics_ideal['r2'] - metrics_hard['r2']
    print(f"   ⚠️ Degradación R²: -{r2_drop:.4f}")

    # Guardar para tabla final
    comparison_results.append({
        'Model': name,
        'Params': f"{sum(p.numel() for p in model.parameters()):,}",
        # Software
        'SW MSE': metrics_ideal['mse'],
        'SW R²': metrics_ideal['r2'],
        # Hardware
        'HW MSE': metrics_hard['mse'],
        'HW R²': metrics_hard['r2'],
        # Delta
        'Gap MSE': mse_gap,
        'Gap R²': r2_drop
    })

print("\n" + "="*80)
print(" RESUMEN FINAL DE IMPLEMENTACIÓN")
print("="*80)

df_results = pd.DataFrame(comparison_results)
# Formato bonito para visualizar en el notebook
print(df_results.to_string(index=False, float_format=lambda x: "{:.6f}".format(x)))

# Guardar a CSV por si acaso
df_results.to_csv('benchmark_hardware_comparison.csv', index=False)
print("\n✅ Benchmark completo guardado en 'benchmark_hardware_comparison.csv'")

In [ ]:
df = pd.DataFrame(results)

# Ordenar por R² (mejor primero)
df_sorted = df.copy()
df_sorted['R2_float'] = df_sorted['Test R²'].astype(float)
df_sorted = df_sorted.sort_values('R2_float', ascending=False)
df_sorted = df_sorted.drop('R2_float', axis=1)

print("\n" + "="*80)
print("COMPARACIÓN FINAL: LSTM vs TCN vs RNN - TEST SET")
print("="*80)
print(df_sorted.to_string(index=False))
print("="*80)
print("\nNOTAS:")
print("- Evaluados en el MISMO test set (14,907 muestras, 15% del dataset)")
print("- Test set NUNCA visto durante entrenamiento")
print("- Split idéntico: benchmark_split_indices.pt")
print("- Métricas comparables directamente")
print("="*80)

# Guardar
comparison_results = {
    'results': results,
    'test_set_size': len(split_indices['test']),
    'models_compared': list(models.keys())
}
torch.save(comparison_results, 'benchmark_comparison.pt')
print("\n✅ Resultados guardados: benchmark_comparison.pt")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def compare_models_on_sample(models_dict, test_loader, device, sample_idx=0):
    """
    Compara los 3 modelos en la misma muestra del test set.
    """
    # Obtener batch específico
    for i, (x, y) in enumerate(test_loader):
        if i == sample_idx:
            x_batch, y_batch = x.to(device), y.to(device)
            break

    # Tomar primera muestra del batch
    audio_in = x_batch[0, :, 0].cpu().numpy()
    y_true = y_batch[0].cpu().numpy().flatten()
    time = np.arange(len(audio_in))

    # Predecir con cada modelo
    predictions = {}
    metrics = {}

    for name, model in models_dict.items():
        model.eval()
        with torch.no_grad():
            y_pred = model(x_batch)

        y_pred_np = y_pred[0].cpu().numpy().flatten()
        predictions[name] = y_pred_np

        mse = np.mean((y_true - y_pred_np)**2)
        mae = np.mean(np.abs(y_true - y_pred_np))
        metrics[name] = {'mse': mse, 'mae': mae}

    # Visualización
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))

    # Plot 1: Señales
    axes[0].plot(time, audio_in, label='Input', alpha=0.4, linewidth=1, color='gray')
    axes[0].plot(time, y_true, label='Ground Truth', alpha=0.9, linewidth=2, color='black')

    colors = {'LSTM': 'blue', 'TCN': 'green', 'RNN': 'red'}
    for name, pred in predictions.items():
        axes[0].plot(time, pred, label=f'{name} (R²={metrics[name]["mse"]:.6f})',
                     alpha=0.7, linewidth=1.5, linestyle='--', color=colors.get(name, 'purple'))

    axes[0].set_xlabel('Time (samples)')
    axes[0].set_ylabel('Amplitude')
    axes[0].set_title(f'Model Comparison - Test Sample (Batch {sample_idx})')
    axes[0].legend(loc='best')
    axes[0].grid(alpha=0.3)

    # Plot 2: Errores
    for name, pred in predictions.items():
        error = y_true - pred
        axes[1].plot(time, error, label=f'{name} error (MAE={metrics[name]["mae"]:.6f})',
                     alpha=0.7, linewidth=1, color=colors.get(name, 'purple'))

    axes[1].axhline(0, color='black', linestyle='--', linewidth=0.8)
    axes[1].set_xlabel('Time (samples)')
    axes[1].set_ylabel('Error (True - Pred)')
    axes[1].set_title('Prediction Errors')
    axes[1].legend(loc='best')
    axes[1].grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'comparison_sample_{sample_idx}.png', dpi=150, bbox_inches='tight')
    plt.show()

    # Tabla de métricas
    print(f"\n{'='*60}")
    print(f"METRICS - Test Sample (Batch {sample_idx})")
    print(f"{'='*60}")
    for name, m in metrics.items():
        print(f"{name:10s} | MSE: {m['mse']:.8f} | MAE: {m['mae']:.6f}")
    print(f"{'='*60}\n")

# Comparar en 3 muestras diferentes
print("Comparando modelos en 3 muestras del test set...\n")

compare_models_on_sample(models, test_loader, device, sample_idx=0)
compare_models_on_sample(models, test_loader, device, sample_idx=50)
compare_models_on_sample(models, test_loader, device, sample_idx=100)
